In [1]:
import pickle
from tensorflow.keras.models import load_model
import pandas as pd

load all trained model

In [2]:
## load model
model = load_model('model.h5')

##Load encoder
with open('encoder_label.pkl', 'rb') as file:
  label_encoder = pickle.load(file)

with open('encoder_one_hot_geography.pkl', 'rb') as file:
  one_hot_geography_encoder = pickle.load(file)

with open('scaler_standard.pkl', 'rb') as file:
  standard_scaler = pickle.load(file)

In [3]:
##Example input data

input_data = {
  'CreditScore': 600,
  'Geography': 'France',
  'Gender':	'Male',
  'Age':	40,
  'Tenure':	3,
  'Balance':	60000,
  'NumOfProducts':	2,
  'HasCrCard':	1,
  'IsActiveMember':	1,
  'EstimatedSalary': 50000
}

In [4]:
## geo encoder
geo_encoded = one_hot_geography_encoder.transform([[input_data['Geography']]]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_geography_encoder.get_feature_names_out(['Geography']))

geo_encoded_df

/Users/sumeetjadhav/miniconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df = pd.DataFrame([input_data])

In [6]:
input_df = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)
input_df.drop(['Geography'], axis=1, inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [7]:
## label encoder for gender

input_df['Gender'] = label_encoder.transform(input_df['Gender'])

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
## scaling the transform

input_scaled = standard_scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

### do the prediction

In [11]:
prediction = model.predict(input_scaled)

prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[0.04321135]], dtype=float32)

In [13]:
prediction_probability = prediction[0][0]
prediction_probability

0.04321135

In [14]:
if(prediction_probability > 0.5):
  print('The customer is likely to churn')
else:
  print('The customer is not likely to churn')

The customer is not likely to churn
